In [11]:
user_search = 'spaghetti'

## Set up Environment

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

### Does this need to be run anymore??

In [9]:


model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [4]:
torch.save(model, './Models/bert_sentiment_model')

### Load Model

In [2]:
model = torch.load('./Models/bert_sentiment_model')
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Verify BERT Model Works

In [3]:
tokens = tokenizer.encode('The font sizes and the amount of white space is just amazing for a ux designer.', return_tensors='pt')
result = model(tokens)
rating = int(torch.argmax(result.logits)) + 1
rating

5

### Evaluate Recipes Using Comments

In [36]:
recipes = pd.read_csv('./recipes.csv')

In [37]:
from ast import literal_eval
recipes['Comments'] = recipes['Comments'].apply(literal_eval)
# recipes['Ingredients'] = recipes['Ingredients'].apply(literal_eval)


In [38]:
def get_rating(comment):
    tokens = tokenizer.encode(comment, return_tensors='pt')
    result = model(tokens)
    rating = int(torch.argmax(result.logits)) + 1
    return rating

In [40]:
avg_rating = [0] * len(recipes.index)
for i in range(len(recipes.index)):
    print(i)
    comments = recipes['Comments'].iloc[i]
    ratings = []
    for c in comments:
#         print(c)
        if c == '':
            continue
#         elif len(c) > 512:
            # split into smaller parts
#             print(c)
        ratings.append(get_rating(c))
    avg_rating[i] = (sum(ratings)/len(ratings))
    
recipes['Rating'] = avg_rating
    
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [44]:
recipes

,Name,Time,Ingredients,Comments,Link,Rating
0,ABC,10 minutes,Nothing,"[check, check, check]",www.blah.com,5.000000
1,Chicken Paprikash,1 hr 20 min,"['1 pound carrots, roughly chopped', '1/3 cup ...","[, , , I follow this all the time minus the ce...",http://www.foodnetwork.com/recipes/chicken-pap...,4.000000
2,Filipino Chicken Adobo,2 hr,"['4 to 5 pounds chicken thighs', '1/2 cup whit...","[Fantastic recipe, I moved to the Philippines ...",http://www.foodnetwork.com/recipes/filipino-ch...,4.500000
3,Chicken Florentine Style,30 min,"['4 boneless skinless chicken breasts', 'Salt ...",[TRY THIS. I've been making this recipe for ye...,http://www.foodnetwork.com/recipes/giada-de-la...,4.860000
4,Chicken Cacciatore,55 min,"['4 chicken thighs', '2 chicken breasts with s...","[If you can't follow the recipe, I'm not inter...",http://www.foodnetwork.com/recipes/giada-de-la...,4.520000
5,Chicken Tortilla Soup,35 min,"['2 tablespoons vegetable oil', '1 small onion...","[, , Very bland. Wish it included more seasoni...",http://www.foodnetwork.com/recipes/chicken-tor...,4.000000
6,Chicken Enchiladas,1 hr 15 min,"['3 tablespoons vegetable oil', '1 1/2 pounds ...","[I followed this recipe exactly, and it was DE...",http://www.foodnetwork.com/recipes/tyler-flore...,4.100000
7,Garlic Chicken and Potatoes,40 min,"['2 pounds small red-skinned potatoes, quarter...",[I loved this recipe. I took the advice of dou...,http://www.foodnetwork.com/recipes/food-networ...,4.300000
8,Chicken Spaghetti,1 hr 40 min,"['1 whole raw chicken, cut into 8 pieces', '1 ...","[I made this yesterday, my husband loved it!! ...",http://www.foodnetwork.com/recipes/ree-drummon...,4.500000
9,"Chicken Cauliflower ""Fried Rice""",20 min,"['1 medium head cauliflower, stem removed', '3...",[I used Green Giant's Cauliflower Rice (steam ...,http://www.foodnetwork.com/recipes/katie-lee/c...,4.460000


In [41]:
recipes.drop(index=recipes.index[0], 
        axis=0, 
        inplace=True)

In [52]:
def parse_time(cook_time):
    minutes = 0
    
    if 'hr' in cook_time:
        time_split = cook_time.split('hr')
        hour = int(time_split[0])
        minutes += int(time_split[0])*60
        cook_time = time_split[1]
        
    if 'min' in cook_time:
        time_split = cook_time.split('min')
        minutes += int(time_split[0])        
    return minutes

In [50]:
parse_time('1 hr')

60

In [53]:
recipes['Time'] = recipes['Time'].apply(lambda x: parse_time(x))

In [56]:
recipes = recipes.sort_values(['Rating', 'Time'], ascending=[False, True]).reset_index(drop=True)


,Name,Time,Ingredients,Comments,Link,Rating
0,Chimichurri Chicken Marinade,10,"['1 cup packed fresh parsley leaves', '2 table...","[, , , I have made this dish over 10 times sin...",http://www.foodnetwork.com/recipes/food-networ...,5.000000
1,Chicken Bolognese,100,"['2 tablespoons olive oil', '1 pound ground ch...",[Can you please put the nutritional informatio...,http://www.foodnetwork.com/recipes/katie-lee/c...,4.960000
2,Chicken Florentine Style,30,"['4 boneless skinless chicken breasts', 'Salt ...",[TRY THIS. I've been making this recipe for ye...,http://www.foodnetwork.com/recipes/giada-de-la...,4.860000
3,Chicken Mozzarella Pasta,16,"['1 pound penne or rigatoni', '2 tablespoons o...",[We made this tonight and just loved it. Hubb...,http://www.foodnetwork.com/recipes/ree-drummon...,4.800000
4,Roman-Style Chicken,60,"['4 skinless chicken breast halves, with ribs'...",[Amazing dish. I tease my wife that Giada is ...,http://www.foodnetwork.com/recipes/giada-de-la...,4.700000
5,White Chicken Chili,55,"['2 (14.5-ounce) cans white beans', '1 tablesp...",[Been making this for years and love it! Usual...,http://www.foodnetwork.com/recipes/patrick-and...,4.640000
6,Grandma’s Smothered Chicken,90,"['4 tablespoons canola oil', '6 medium chicken...",[Fabulous and easy! I’ve never tried a cinnamo...,http://www.foodnetwork.com/recipes/grandmas-sm...,4.568182
7,Chile-Chicken Posole,30,"['1 pound skinless, boneless chicken breasts, ...","[So good and so easy, and unlike some of the p...",http://www.foodnetwork.com/recipes/food-networ...,4.520000
8,Chicken Cacciatore,55,"['4 chicken thighs', '2 chicken breasts with s...","[If you can't follow the recipe, I'm not inter...",http://www.foodnetwork.com/recipes/giada-de-la...,4.520000
9,Chicken Spaghetti,100,"['1 whole raw chicken, cut into 8 pieces', '1 ...","[I made this yesterday, my husband loved it!! ...",http://www.foodnetwork.com/recipes/ree-drummon...,4.500000


In [57]:
recipes.to_csv('./Data/rated_recipes.csv', index = False, header=True)